In [1]:
import os
import pandas as pd
import random
from tqdm import tqdm
import re

print(len(os.listdir('VF_Main_16kHz/')))

6246


In [2]:
docker_path = '/mnt/data/'

In [10]:
all_spk = {}; anonymous = 0

for folder in tqdm(os.listdir('VF_Main_16kHz/')):
    spk = folder[:folder.find('-')]
    if spk == 'anonymous' or spk == 'jaiger': anonymous +=  1
    if spk not in all_spk:
        all_spk[spk] = 1
    else:
        all_spk[spk] += 1
print(len(all_spk))
print(anonymous)

100%|██████████| 6246/6246 [00:00<00:00, 969277.15it/s]

1235
2164


In [60]:
all_spk = list(all_spk.keys())
selected_spk = []

def select_spk(num_spk):
    spk = []
    for i in range(num_spk):
        rand_spk = random.choice(all_spk)
        while rand_spk in selected_spk:
            rand_spk = random.choice(all_spk)
        spk.append(rand_spk)
        selected_spk.append(rand_spk)
    return spk

In [61]:
train_spk = select_spk(100)
test_spk = select_spk(20)
len(set(train_spk).intersection(test_spk)) == 0

True

In [62]:
prompt_file = 'prompts-original'

def get_df(select_spk):
    spk_id = []
    utt_id = []
    text = []
    wav_path = []
    no_wav = 0
    no_prompt = 0
    for folder in tqdm(os.listdir('VF_Main_16kHz/')):
        speaker = folder[:folder.find('-')]
        if speaker not in select_spk:
            continue
        waves = []
        if 'wav' not in os.listdir(os.path.join('VF_Main_16kHz/', folder)):
            no_wav += 1 
            continue
        if prompt_file not in os.listdir(os.path.join('VF_Main_16kHz/', folder, 'etc/')):
            no_prompt += 1
            continue
        
        prompts = [line.rstrip('\n') for line in open(os.path.join('VF_Main_16kHz/', folder, 'etc/', prompt_file))]
        #print("test")
        for prompt in prompts:
            if prompt[:prompt.find(' ')] == '':
                continue
            start_idx = prompt.find('/') + 1 if '/' in prompt[:prompt.find(' ')] else 0
            waves.append(prompt[start_idx:prompt.find(' ')])
            #if len(prompt[start_idx:prompt.find(' ')]) > 11:
                #print(prompt[start_idx:prompt.find(' ')])
                #print(prompt)
            clean_text = prompt[prompt.find(' '):].strip().upper()
            clean_text = re.sub(r"[^\w\d's]+",' ',clean_text)
            #print(clean_text)
            #print(clean_text[clean_text.find(' '):])
            text.append(clean_text)

        for wav in waves:
            local_path = os.path.join('VF_Main_16kHz/', folder, 'wav/', wav + '.wav')
            if not os.path.isfile(local_path):
                print(local_path)
            wav_path.append(os.path.join(docker_path, local_path))
            utt_id.append(folder + '_' + wav)
            spk_id.append(speaker)
    print("no prompt: ",no_prompt)
    print("no wav: ",no_wav)
    print('All folders:', len(os.listdir('VF_Main_16kHz/')))
    print('Other extensions:', no_wav)
    print('No prompts-original:', no_prompt)
    print(len(spk_id), len(utt_id), len(text), len(wav_path))
    return pd.DataFrame({'spk_id': spk_id, 'utt_id': utt_id, 'text': text, 'wav_path': wav_path})  

In [63]:
train_df = get_df(train_spk)
train_df = train_df.sort_values(by=['spk_id', 'utt_id'])
train_df = train_df.reset_index(drop=True)
train_df.head()

100%|██████████| 6246/6246 [00:00<00:00, 14439.12it/s]

no prompt:  0
no wav:  1
All folders: 6246
Other extensions: 1
No prompts-original: 0
5790 5790 5790 5790


,spk_id,utt_id,text,wav_path
0,Aaron,Aaron-20080318-kdl_b0019,HIS SLIM HANDS GRIPPED THE EDGES OF THE TABLE,/mnt/data/VF_Main_16kHz/Aaron-20080318-kdl/wav...
1,Aaron,Aaron-20080318-kdl_b0020,HE MADE NO REPLY AS HE WAITED FOR WHITTEMORE T...,/mnt/data/VF_Main_16kHz/Aaron-20080318-kdl/wav...
2,Aaron,Aaron-20080318-kdl_b0021,PHILIP DROPPED BACK INTO HIS CHAIR,/mnt/data/VF_Main_16kHz/Aaron-20080318-kdl/wav...
3,Aaron,Aaron-20080318-kdl_b0022,IF I WAS OUT OF THE GAME IT WOULD BE EASILY MADE,/mnt/data/VF_Main_16kHz/Aaron-20080318-kdl/wav...
4,Aaron,Aaron-20080318-kdl_b0023,MACDOUGALL MY ENGINEER BELIEVES IT,/mnt/data/VF_Main_16kHz/Aaron-20080318-kdl/wav...


In [64]:
test_df = get_df(test_spk)
test_df = test_df.sort_values(by=['spk_id', 'utt_id'])
test_df = test_df.reset_index(drop=True)
test_df.head()

100%|██████████| 6246/6246 [00:00<00:00, 55453.86it/s]

no prompt:  10
no wav:  71
All folders: 6246
Other extensions: 71
No prompts-original: 10
732 732 732 732


,spk_id,utt_id,text,wav_path
0,Blenderkitty,Blenderkitty-20090615-qcv_b0419,YOUR FATHER'S FIFTH COMMAND HE NODDED,/mnt/data/VF_Main_16kHz/Blenderkitty-20090615-...
1,Blenderkitty,Blenderkitty-20090615-qcv_b0420,ON OCCASION ON THIS TRAVERSE THE CAPE VERDE IS...,/mnt/data/VF_Main_16kHz/Blenderkitty-20090615-...
2,Blenderkitty,Blenderkitty-20090615-qcv_b0421,SHE IS ESSENTIALLY THE LIFE GIVING LIFE CONSER...,/mnt/data/VF_Main_16kHz/Blenderkitty-20090615-...
3,Blenderkitty,Blenderkitty-20090615-qcv_b0422,THIS WAS WHEN THE EXPLOSION OCCURRED,/mnt/data/VF_Main_16kHz/Blenderkitty-20090615-...
4,Blenderkitty,Blenderkitty-20090615-qcv_b0423,ALSO AT REGULAR INTERVALS HE WOULD MUTTER,/mnt/data/VF_Main_16kHz/Blenderkitty-20090615-...


In [65]:
# Note : *************************************this code is "append mode"
def data_prep(df, mode):
    os.makedirs(os.path.join('data', mode), exist_ok=True)
    with open(os.path.join('data', mode, 'text'), 'a') as file:
        for _, row in df.iterrows():
            content = row['utt_id'] + ' ' + row['text'] + '\n'
            file.write(content)
    with open(os.path.join('data', mode, 'wav.scp'), 'a') as file:
        for _, row in df.iterrows():
            content = row['utt_id'] + ' ' + row['wav_path'] + '\n'
            file.write(content)
    with open(os.path.join('data', mode, 'utt2spk'), 'a') as file:
        for _, row in df.iterrows():
            content = row['utt_id'] + ' ' + row['spk_id'] + '\n'
    with open(os.path.join('data', mode, 'puretext.txt'), 'a') as file:
        for _, row in df.iterrows():
            content = row['text'] + '\n'
            file.write(content)

In [66]:
data_prep(train_df, 'train')

In [67]:
data_prep(test_df, 'test')